In [1]:
# !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
# !unzip -q kagglecatsanddogs_3367a.zip
# !ls PetImages

In [2]:
import re
from glob import glob

import numpy as np
from scipy import stats
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
image_size = (32, 32)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [4]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [5]:
model_re = re.compile('save_at_(\d+).h5')

In [6]:
models = sorted(glob('models/save_at/save_at_*.h5'), key=lambda x: int(model_re.findall(x)[0]))

In [7]:
model = keras.models.load_model(models[-1])

In [8]:
class SeModel:
    def __init__(self, model_pathes):
        self.model = list(map(keras.models.load_model, model_pathes))
        
    def predict(self, f):
        return np.concatenate(list(map(lambda m: m.predict(f), self.model)), axis=1)

In [9]:
indeces = 25, 30, 35, 38, 40, 43, 45, 47, 48, 49
se_model = SeModel([models[i] for i in indeces])

In [10]:
# se_model = SeModel(models[-10:])

In [11]:
label = list()
predict = None
_se_predict = None

for i, j in val_ds:
    if predict is None:
        predict = model.predict(i)
        _se_predict = se_model.predict(i)
    else:
        predict = np.concatenate([predict, model.predict(i)])
        _se_predict = np.concatenate([_se_predict, se_model.predict(i)])
    label.extend(list(map(int, j)))

In [12]:
se_predict = np.array(stats.mode(_se_predict>0.5, axis=1))[0, :, :]
accuracy_score(se_predict, label)

0.8771892353695002

In [13]:
accuracy_score(list(map(lambda x:x>0.5, predict)), label)

0.8665100384451089